# Workbench Client

In [ ]:
import os
import json
import pyvista as pv
from ansys.api.workbench.v0.launch_workbench import LaunchWorkbench
from ansys.mechanical.core import launch_mechanical

In [ ]:
# launch Workbench service on a remote host
# host = "remote_name_or_ip"
# username = "your_username"
# password = "your_password"
# wb = LaunchWorkbench(host, username, password)

# launch Workbench on the local machine
host = 'localhost'
wb = LaunchWorkbench()

In [ ]:
# upload a couple of input files
wb.upload_file_from_example_repo("2pipes.agdb", "2pipes")
wb.upload_file("solve.py")

In [ ]:
# run a Workbench script to define the project and load geometry
output = wb.run_script_file('geometry.wbjn')
print(output)

In [ ]:
# start PyMechanical service then create a local PyMechanical client
mech_port = wb.run_script_file('start_pymech.wbjn')
print("mechanical grpc started on port " + str(mech_port))
mechanical = launch_mechanical(start_instance=False, ip=host, port=mech_port)
print(mechanical.project_directory)

In [ ]:
# run a Mechanical python script via PyMechanical to mesh/solve the model
with open (os.path.join(wb.client_workdir, "solve.py")) as sf:
    mech_script = sf.read()
mech_output = mechanical.run_python_script(mech_script)
print(mech_output)

In [ ]:
# download a couple of output files after copying them to working directory
wb.run_script_file('copy_files.wbjn')
wb.download_file("solve.out")
wb.download_file("deformation.png")

In [ ]:
# display the content of the solver output
with open(os.path.join(wb.client_workdir, "solve.out"), "r") as f:
    print(f.read())

In [ ]:
# plot the deformation result generated by the solver
pl = pv.Plotter()
pl.add_background_image(os.path.join(wb.client_workdir, "deformation.png"))
pl.show(jupyter_backend='static')

In [ ]:
# shutdown the Workbench service
wb.shutdown()